# AM 205 - Midterm

## By Jonathan Guillotte-Blouin

### Question 1: A stencil based on least squares

#### a) Calculate the coefficients $a_{-1}$, $a_0$, $a_1$, $a_2$, and $a_3$ to make a fourth-order accurate finite-difference formula.

To do this, we will use the Lagrange interpolant approach.

$$ \mathcal{L_{-1}}(z) = \frac{(z-kh)(z-kh-h)(z-kh-2h)(z-kh-3h)}{(-h)(-2h)(-3h)(-4h)} = \frac{(z-kh)(z-kh-h)(z-kh-2h)(z-kh-3h)}{24h^4}$$

Assume $\mathcal{L'_x(z)} = \frac{\partial L_x(z)}{\partial z}$.

$$ \mathcal{L'_{-1}}(0) = -\frac{2h^3(2k^3 +9k^2+11k+3)}{24h^4} = -\frac{2k^3 +9k^2+11k+3}{12h}$$

$$ \mathcal{L_{0}}(z) = \frac{(z-kh+h)(z-kh-h)(z-kh-2h)(z-kh-3h)}{h(-h)(-2h)(-3h)} = \frac{(z-kh+h)(z-kh-h)(z-kh-2h)(z-kh-3h)}{-6h^4}$$

$$ \mathcal{L'_{0}}(0) = -\frac{h^3(4k^3 +15k^2+10k-5)}{-6h^4} = \frac{4k^3 +15k^2+10k-5}{6h}$$


$$ \mathcal{L_{1}}(z) = \frac{(z-kh+h)(z-kh)(z-kh-2h)(z-kh-3h)}{(2h)h(-h)(-2h)} = \frac{(z-kh+h)(z-kh)(z-kh-2h)(z-kh-3h)}{4h^4}$$

$$ \mathcal{L'_{1}}(0) = -\frac{2h^3(2k^3 +6k^2+k-3)}{4h^4} = -\frac{2k^3 +6k^2+k-3}{2h}$$



$$ \mathcal{L_{2}}(z) = \frac{(z-kh+h)(z-kh)(z-kh-h)(z-kh-3h)}{(3h)(2h)h(-h)} = \frac{(z-kh+h)(z-kh)(z-kh-2h)(z-kh-3h)}{-6h^4}$$

$$ \mathcal{L'_{2}}(0) = \frac{h^3(-4k^3 -9k^2+2k+3)}{-6h^4} = -\frac{-4k^3 -9k^2+2k+3}{6h}$$


$$ \mathcal{L_{3}}(z) = \frac{(z-kh+h)(z-kh)(z-kh-h)(z-kh-2h)}{(4h)(3h)(2h)h} = \frac{(z-kh+h)(z-kh)(z-kh-h)(z-kh-2h)}{24h^4}$$

$$ \mathcal{L'_{3}}(0) = \frac{-2h^3(2k^3 +3k^2-k-1)}{24h^4} = -\frac{2k^3 +3k^2-k-1}{12h}$$

$$ l'(z) = \mathcal{L'_{-1}}(z) \, f_{k-1} + \mathcal{L'_{0}}(z) \, f_k + \mathcal{L'_{1}}(z) \, f_{k+1} + \mathcal{L'_{2}}(z) \, f_{k+2} + \mathcal{L'_{3}}(z) \, f_{k+3} $$

$$ f'_{diff1}(x_k) = l'(0) = \frac{-\frac{2k^3 +9k^2+11k+3}{12} \, f_{k-1} + \frac{4k^3 +15k^2+10k-5}{6} \, f_k -\frac{2k^3 +6k^2+k-3}{2} \, f_{k+1} - \frac{-4k^3 -9k^2+2k+3}{6} \, f_{k+2} - \frac{2k^3 +3k^2-k-1}{12} \, f_{k+3}}{h} $$